In [1]:
import pandas as pd
import string
import nltk
import math
import tensorflow as tfl
import numpy as np
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex as re

pd.set_option('display.max_columns', None)

In [2]:
file = pd.read_csv("scrapping_BTC_comments.csv")
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So /u/spez how much longer are you going to ig...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised /r/bitcoin mod...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,/u/spez I once shied away from replying to you...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$10 /u/tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from /u/4n4n4 got the double dash...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


## Data Cleaning

In [3]:
file["body"]

0      So /u/spez how much longer are you going to ig...
1      Wow this is huge. I'm surprised /r/bitcoin mod...
2                                          Holy shit....
3      I'm a bot, *bleep*, *bloop*. Someone has linke...
4      /u/spez I once shied away from replying to you...
5      I don't know whether I should feel enraged by ...
6              Great detective work!\r\n\r\n$10 /u/tippr
7      Wow, epic, standing ovation! This confirms wha...
8      This comment from /u/4n4n4 got the double dash...
9      If I remember correctly Gregory Maxwell often ...
10     Paging u/adam3us, what do you feel about a C-l...
11     Smoke everywhere.  /u/bashco clearly had knowl...
12     The technical term is "false flag".\r\n\r\n>  ...
13                                               TL;DR? 
14                       Initiate Operation Dragonslayer
15     Thank you for doing this. More evidence to col...
16     The most telling aspect of this is the solid l...
17     I don't know who this ce

### Remove all "/u/", "/r/", "/r", "\n"

In [4]:
for i in range(len(file["body"])):
    file["body"][i] = re.sub(r"(\/\w\/|\w\/)", "", file["body"][i])
    file["body"][i] = re.sub(r"\\n?t?", "", file["body"][i])
    file["body"][i] = re.sub(r"[0-9]*", "", file["body"][i])

file.head(20)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,created_utc,body,score,permalink,date
0,1.511280e+09,So spez how much longer are you going to ignor...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge. I'm surprised bitcoin mods w...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit....,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"I'm a bot, *bleep*, *bloop*. Someone has linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,spez I once shied away from replying to you re...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I don't know whether I should feel enraged by ...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work!\r\n\r\n$ tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"Wow, epic, standing ovation! This confirms wha...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"This comment from nn got the double dash, no s...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove punctuation

In [5]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [6]:
file['body'] = file['body'].apply(lambda x: remove_punctuation(x))
file

,created_utc,body,score,permalink,date
0,1.511280e+09,So spez how much longer are you going to ignor...,2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,Wow this is huge Im surprised bitcoin mods wer...,631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,Holy shit,720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,Im a bot bleep bloop Someone has linked to thi...,244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,spez I once shied away from replying to you re...,34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,I dont know whether I should feel enraged by w...,212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,Great detective work\r\n\r\n tippr,127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,Wow epic standing ovation This confirms what w...,164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,This comment from nn got the double dash no sp...,23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,If I remember correctly Gregory Maxwell often ...,115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove short comments (in terms of words)

In [7]:
tokenizer = RegexpTokenizer(r'\w+')

In [8]:
file['body'] = file['body'].apply(lambda x : tokenizer.tokenize(x.lower()))
file.head(20)

,created_utc,body,score,permalink,date
0,1.511280e+09,"[so, spez, how, much, longer, are, you, going,...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, is, huge, im, surprised, bitcoin, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,"[holy, shit]",720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"[im, a, bot, bleep, bloop, someone, has, linke...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, i, once, shied, away, from, replying, t...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[i, dont, know, whether, i, should, feel, enra...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, nn, got, the, double, da...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[if, i, remember, correctly, gregory, maxwell,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


In [9]:
for i in range(len(file['body'])):
    for word in file['body'][i]:
        if len(word) < 3:
            file['body'][i].remove(word)
        
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, how, much, longer, are, you, going, ign...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, this, huge, surprised, bitcoin, mods, we...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,"[holy, shit]",720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"[a, bot, bleep, bloop, someone, has, linked, t...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, once, shied, away, from, replying, you,...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, should, feel, enraged, w...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, this, confirms,...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[this, comment, from, got, the, double, dash, ...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove Stopwords

In [10]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [11]:
file['body'] = file['body'].apply(lambda x : remove_stopwords(x))
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,"[holy, shit]",720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


### Remove Reddit links

In [12]:
def remove_links(text):
    words = [w for w in text if not len(w) > 20]
    return words

In [13]:
file['body'] = file['body'].apply(lambda x : remove_links(x))
file

,created_utc,body,score,permalink,date
0,1.511280e+09,"[spez, much, longer, going, ignore, blatant, c...",2290,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:12
1,1.511281e+09,"[wow, huge, surprised, bitcoin, mods, sloppy, ...",631,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:11:11
2,1.511278e+09,"[holy, shit]",720,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:32:00
3,1.511279e+09,"[bot, bleep, bloop, someone, linked, thread, a...",244,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 16:46:08
4,1.511291e+09,"[spez, shied, away, replying, reddit, admins, ...",34,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 20:00:57
5,1.511280e+09,"[dont, know, whether, feel, enraged, happened,...",212,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:06:14
6,1.511280e+09,"[great, detective, work, tippr]",127,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:03:40
7,1.511281e+09,"[wow, epic, standing, ovation, confirms, suspe...",164,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 17:17:23
8,1.511289e+09,"[comment, got, double, dash, space, talk, usin...",23,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:37:51
9,1.511289e+09,"[remember, correctly, gregory, maxwell, often,...",115,/r/btc/comments/7eil12/evidence_that_the_mods_...,2017-11-21 19:29:34


## Data analysis

### Unique words

In [14]:
uniqueWords = []

for sentence in file["body"]:
    words = [word for word in sentence]
    for word in words:
        if word not in uniqueWords:
            uniqueWords.append(word)

uniqueWords

['spez',
 'much',
 'longer',
 'going',
 'ignore',
 'blatant',
 'censorship',
 'manipulation',
 'bitcoin',
 'mods',
 'wow',
 'huge',
 'surprised',
 'sloppy',
 'able',
 'uncover',
 'vote',
 'botting',
 'means',
 'involved',
 'hacking',
 'wonder',
 'actually',
 'whats',
 'crazy',
 'connection',
 'nullc',
 'shows',
 'clearly',
 'blockstream',
 'working',
 'together',
 'manipulate',
 'entire',
 'market',
 'dumbfounded',
 'would',
 'really',
 'dont',
 'see',
 'reddit',
 'admins',
 'take',
 'action',
 'user',
 'accounts',
 'edit',
 'highly',
 'recommend',
 'message',
 'look',
 'holy',
 'shit',
 'bot',
 'bleep',
 'bloop',
 'someone',
 'linked',
 'thread',
 'another',
 'place',
 'againstastroturfing',
 'evidence',
 'rbitcoin',
 'may',
 'attack',
 'bcore',
 'berniesanders',
 'bgold',
 'information',
 'bitcoinexposed',
 'bitcoinuncensored',
 'bitcoincash',
 'bitcoindiscussion',
 'gmaxwell',
 'bitcoinmtl',
 'bitcoinno',
 'bitcoinxt',
 'blockstreams',
 'cto',
 'greg',
 'maxwell',
 'caught',
 'using

In [15]:
numOfWords = dict.fromkeys(uniqueWords, 0)
numOfWords

{'spez': 0,
 'much': 0,
 'longer': 0,
 'going': 0,
 'ignore': 0,
 'blatant': 0,
 'censorship': 0,
 'manipulation': 0,
 'bitcoin': 0,
 'mods': 0,
 'wow': 0,
 'huge': 0,
 'surprised': 0,
 'sloppy': 0,
 'able': 0,
 'uncover': 0,
 'vote': 0,
 'botting': 0,
 'means': 0,
 'involved': 0,
 'hacking': 0,
 'wonder': 0,
 'actually': 0,
 'whats': 0,
 'crazy': 0,
 'connection': 0,
 'nullc': 0,
 'shows': 0,
 'clearly': 0,
 'blockstream': 0,
 'working': 0,
 'together': 0,
 'manipulate': 0,
 'entire': 0,
 'market': 0,
 'dumbfounded': 0,
 'would': 0,
 'really': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admins': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'accounts': 0,
 'edit': 0,
 'highly': 0,
 'recommend': 0,
 'message': 0,
 'look': 0,
 'holy': 0,
 'shit': 0,
 'bot': 0,
 'bleep': 0,
 'bloop': 0,
 'someone': 0,
 'linked': 0,
 'thread': 0,
 'another': 0,
 'place': 0,
 'againstastroturfing': 0,
 'evidence': 0,
 'rbitcoin': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesanders': 0,
 'bgold': 0,
 'inform

In [16]:
for i in range(len(file)):
    try :
        for word in file["body"][i]:
            numOfWords[word] += 1
    except KeyError:
        i += 1
        
numOfWords

{'spez': 7,
 'much': 11,
 'longer': 2,
 'going': 10,
 'ignore': 1,
 'blatant': 2,
 'censorship': 10,
 'manipulation': 40,
 'bitcoin': 66,
 'mods': 47,
 'wow': 6,
 'huge': 1,
 'surprised': 3,
 'sloppy': 1,
 'able': 4,
 'uncover': 1,
 'vote': 42,
 'botting': 1,
 'means': 2,
 'involved': 37,
 'hacking': 37,
 'wonder': 2,
 'actually': 5,
 'whats': 6,
 'crazy': 2,
 'connection': 1,
 'nullc': 11,
 'shows': 3,
 'clearly': 5,
 'blockstream': 16,
 'working': 4,
 'together': 2,
 'manipulate': 2,
 'entire': 4,
 'market': 2,
 'dumbfounded': 1,
 'would': 16,
 'really': 7,
 'dont': 13,
 'see': 14,
 'reddit': 25,
 'admins': 7,
 'take': 7,
 'action': 2,
 'user': 2,
 'accounts': 4,
 'edit': 2,
 'highly': 3,
 'recommend': 1,
 'message': 1,
 'look': 4,
 'holy': 3,
 'shit': 5,
 'bot': 11,
 'bleep': 1,
 'bloop': 1,
 'someone': 10,
 'linked': 1,
 'thread': 9,
 'another': 5,
 'place': 3,
 'againstastroturfing': 1,
 'evidence': 38,
 'rbitcoin': 50,
 'may': 31,
 'attack': 40,
 'bcore': 1,
 'berniesanders': 1,


### Term Frequency (TF)

In [17]:
def computeTF(file, uniqueWords):
    tf = []
    index = 0
    
    try :
        for sentence in file["body"]:
            tf.append(dict.fromkeys(uniqueWords, 0))
            for word in sentence:
                tf[index][word] += 1 / len(sentence)
            index += 1
    except KeyError :
        print("error")
    return tf

In [18]:
tf = computeTF(file, uniqueWords)
tf[0]

{'spez': 0.1,
 'much': 0.1,
 'longer': 0.1,
 'going': 0.1,
 'ignore': 0.1,
 'blatant': 0.1,
 'censorship': 0.1,
 'manipulation': 0.1,
 'bitcoin': 0.1,
 'mods': 0.1,
 'wow': 0,
 'huge': 0,
 'surprised': 0,
 'sloppy': 0,
 'able': 0,
 'uncover': 0,
 'vote': 0,
 'botting': 0,
 'means': 0,
 'involved': 0,
 'hacking': 0,
 'wonder': 0,
 'actually': 0,
 'whats': 0,
 'crazy': 0,
 'connection': 0,
 'nullc': 0,
 'shows': 0,
 'clearly': 0,
 'blockstream': 0,
 'working': 0,
 'together': 0,
 'manipulate': 0,
 'entire': 0,
 'market': 0,
 'dumbfounded': 0,
 'would': 0,
 'really': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admins': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'accounts': 0,
 'edit': 0,
 'highly': 0,
 'recommend': 0,
 'message': 0,
 'look': 0,
 'holy': 0,
 'shit': 0,
 'bot': 0,
 'bleep': 0,
 'bloop': 0,
 'someone': 0,
 'linked': 0,
 'thread': 0,
 'another': 0,
 'place': 0,
 'againstastroturfing': 0,
 'evidence': 0,
 'rbitcoin': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesanders': 0,
 

### Inverse Data Frequency (IDF)

In [19]:
idfs = dict.fromkeys(uniqueWords, 0)
idfs

{'spez': 0,
 'much': 0,
 'longer': 0,
 'going': 0,
 'ignore': 0,
 'blatant': 0,
 'censorship': 0,
 'manipulation': 0,
 'bitcoin': 0,
 'mods': 0,
 'wow': 0,
 'huge': 0,
 'surprised': 0,
 'sloppy': 0,
 'able': 0,
 'uncover': 0,
 'vote': 0,
 'botting': 0,
 'means': 0,
 'involved': 0,
 'hacking': 0,
 'wonder': 0,
 'actually': 0,
 'whats': 0,
 'crazy': 0,
 'connection': 0,
 'nullc': 0,
 'shows': 0,
 'clearly': 0,
 'blockstream': 0,
 'working': 0,
 'together': 0,
 'manipulate': 0,
 'entire': 0,
 'market': 0,
 'dumbfounded': 0,
 'would': 0,
 'really': 0,
 'dont': 0,
 'see': 0,
 'reddit': 0,
 'admins': 0,
 'take': 0,
 'action': 0,
 'user': 0,
 'accounts': 0,
 'edit': 0,
 'highly': 0,
 'recommend': 0,
 'message': 0,
 'look': 0,
 'holy': 0,
 'shit': 0,
 'bot': 0,
 'bleep': 0,
 'bloop': 0,
 'someone': 0,
 'linked': 0,
 'thread': 0,
 'another': 0,
 'place': 0,
 'againstastroturfing': 0,
 'evidence': 0,
 'rbitcoin': 0,
 'may': 0,
 'attack': 0,
 'bcore': 0,
 'berniesanders': 0,
 'bgold': 0,
 'inform

In [20]:
def computeIDF(idfs, numOfWords, file):
    for x, y in idfs.items():
        count = 0
        for i in range(len(file)):
            try:
                for word in file["body"][i]:
                    if x == word:
                        count += 1
                        break
            except KeyError:
                i += 1
        if count != 0:
            idfs[x] = math.log(len(file) / count)
    return idfs

In [21]:
idfs = computeIDF(idfs, numOfWords, file)
idfs

{'spez': 3.172083663361442,
 'much': 2.920769235080536,
 'longer': 4.42484663185681,
 'going': 3.0385522707369192,
 'ignore': 5.117993812416755,
 'blatant': 4.42484663185681,
 'censorship': 3.7316994512968646,
 'manipulation': 3.7316994512968646,
 'bitcoin': 1.6522579096170285,
 'mods': 2.7200985396183848,
 'wow': 3.3262343431887,
 'huge': 5.117993812416755,
 'surprised': 4.42484663185681,
 'sloppy': 5.117993812416755,
 'able': 4.019381523748645,
 'uncover': 5.117993812416755,
 'vote': 3.5085558999826545,
 'botting': 5.117993812416755,
 'means': 4.42484663185681,
 'involved': 4.019381523748645,
 'hacking': 3.5085558999826545,
 'wonder': 4.42484663185681,
 'actually': 3.5085558999826545,
 'whats': 3.5085558999826545,
 'crazy': 4.42484663185681,
 'connection': 5.117993812416755,
 'nullc': 2.7200985396183848,
 'shows': 4.019381523748645,
 'clearly': 3.7316994512968646,
 'blockstream': 2.5530444549552183,
 'working': 3.7316994512968646,
 'together': 4.42484663185681,
 'manipulate': 4.42484

### TF-IDF

In [22]:
tf[0]['spez']

0.1

In [23]:
def computeTF_IDF(tf, idfs):
    tab = []
    for i in range(len(tf)):
        dico = {}
        for mot in tf[i]:
            dico[mot] = tf[i][mot] * idfs[mot]
        tab.append(dico)
    tfidf = pd.DataFrame(tab)
    return tfidf
            

In [24]:
tfidf = computeTF_IDF(tf, idfs)
tfidf

able  absence  absolutely   account  accounts  accusation  accused  \
0    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
1    0.078811      0.0         0.0  0.000000  0.078811    0.000000      0.0   
2    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
3    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
4    0.000000      0.0         0.0  0.067882  0.000000    0.000000      0.0   
5    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
6    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
7    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
8    0.000000      0.0         0.0  0.057349  0.000000    0.000000      0.0   
9    0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
10   0.000000      0.0         0.0  0.000000  0.000000    0.119023      0.0   
11   0.000000      0.0         0.0  0.175065  0.000000    0.000000      0.0   
12   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
13   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
14   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
15   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
16   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
17   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
18   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
19   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
20   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
21   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
22   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
23   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
24   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
25   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
26   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
27   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
28   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
29   0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
..        ...      ...         ...       ...       ...         ...      ...   
137  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
138  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
139  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
140  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
141  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
142  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
143  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
144  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
145  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
146  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
147  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
148  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
149  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
150  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
151  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
152  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
153  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
154  0.000000      0.0         0.0  0.000000  0.000000    0.000000      0.0   
155  0.000000      0.0         0.0  0.000000  0.000000    0

### Get important words

In [25]:
columns = tfidf.columns.values
words_importance = {}
index = 0
for label in columns:
    words_importance[label] = sum(tfidf[columns[index]])
    index += 1
    
words_importance = sorted(words_importance.items(), key=operator.itemgetter(1), reverse = True)[0:50]
words_importance

[('tippr', 27.214646187215433),
 ('gild', 17.330118232824592),
 ('deleted', 15.462904862845335),
 ('spez', 7.610558980439738),
 ('bitcoin', 7.299384026934222),
 ('work', 6.143410124254372),
 ('tldr', 5.933372725533714),
 ('holyshit', 5.117993812416755),
 ('great', 4.52058466771671),
 ('someone', 4.023000844299347),
 ('rbitcoin', 3.818458789915727),
 ('reddit', 3.6943598674079325),
 ('shit', 3.6261645656366888),
 ('damn', 3.521581238949068),
 ('please', 3.496855549018873),
 ('post', 3.190570362670248),
 ('sticky', 3.0973926422997673),
 ('thank', 3.0449860028398827),
 ('bch', 2.980516285669359),
 ('time', 2.946670923583643),
 ('know', 2.596021069740944),
 ('dr', 2.5589969062083777),
 ('peep', 2.5589969062083777),
 ('pin', 2.5589969062083777),
 ('pls', 2.5589969062083777),
 ('stuff', 2.5589969062083777),
 ('upvotedownvotes', 2.5589969062083777),
 ('like', 2.5561867016582305),
 ('analysis', 2.528483789632463),
 ('big', 2.528483789632463),
 ('mods', 2.5221922501921448),
 ('gold', 2.50556963